## Pipeline Flavor Profile

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import csv

c:\Users\Franz\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


#### 1. MS Data ----- Massbank -----> Molecules

-> Chris

save which intensities belong to which molecule and give each the % 
2 approaches for one molecule with multiple OQs
1.  each gets same % 
2.  divide % by count of OQs per molecule

mol_int : dict where key is the molecule name and value is the intensites from the mass spec data

In [261]:
## example list of moldecules
intensities = [80, 40, 5]
ms_molecules_ex = ['(E)-3-hexenol', "2-dodecenal",'δ-muurolene']
mol_int = {'(E)-3-hexenol': 80, "2-dodecenal": 40 ,'δ-muurolene': 5}

#### 2. Molecules ----- Flavornet -----> OD 
Find ODs by matching molecules with databases -> Flavornet

*CAS_to_odorants.csv* - molecule, CAS, Mol wt, odorant


##### table manipualtion done in R

```r
library(rlang)
library(tidyverse)

CAS_to_odorants <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/CAS_to_mol.txt", sep = "\t", header = FALSE)
OD_to_mol <- read.table("~/Bioinformatik_20.21/Bachelorarbeit/OD_mol.txt", sep = "\t", header = FALSE)

OD_to_mol_sep <- OD_to_mol %>%
  separate_rows(V2, sep = ";\\s*") %>%
  mutate(V2 = trimws(V2))
OD_to_mol_final <- OD_to_mol_sep[, c("V2","V1")]
colnames(OD_to_mol_final) <- c("molecule", "OD")

CAS_to_odorants_sep <- CAS_to_odorants %>%
  separate_rows(V3, sep = ";\\s*") %>%
  mutate(V3 = trimws(V3))
colnames(CAS_to_odorants_sep) <- c("CAS","Mol Wt", "molecule")


merged_df <- merge(CAS_to_odorants_sep, OD_to_mol_final, by = "molecule")
write.csv(merged_df, file = "~/Bioinformatik_20.21/Bachelorarbeit/CAS_mol_OD.csv", row.names = FALSE)
```

In [2]:
mol_to_OD = {}
with open('./data/CAS_mol_OD.csv', 'r',encoding='utf-8') as tabfile:
    reader = csv.reader(tabfile, delimiter=',')
    for row in reader: 
        molecule = row[0]
        CAS = row[1]
        Mol_wt = row[2]
        OD = row[3]
        if molecule in mol_to_OD:
            mol_to_OD[molecule].add(OD)
        else:
            mol_to_OD[molecule]  = set([OD])
        #print(molecule,CAS,Mol_wt, OD)

mol_to_OD : dict where key is the molecule and the descriptors for these molecules are the values

In [3]:
mol_to_OD

{'(-)-carvone': {'mint'},
 '(-)-cis-rose oxide': {'rose', 'sweet'},
 '(-)-cubenol': {'green tea', 'herb', 'spice'},
 '(-)-nootkatone': {'terpentine'},
 '(-)-β-bisabolene': {'balsamic'},
 '(-)-γ-elemene': {'green', 'oil', 'wood'},
 '(+)-(E)-limonene oxide': {'green'},
 '(+)-carvone': {'caraway'},
 '(+)-cis-rose oxide': {'flower', 'green'},
 '(+)-limonene,': {'citrus', 'mint'},
 '(+)-nootkatone': {'grapefruit'},
 '(+)-α-phellandrene': {'dill'},
 '(2S)-2-methyl-ethyl ester-butanoic acid': {'apple'},
 '(E)-2-dodecen-1-ol': {'oil'},
 '(E)-2-hexenal': {'green', 'leaf'},
 '(E)-2-hexenoic acid': {'fat', 'must'},
 '(E)-2-hexenol': {'green', 'leaf', 'walnut'},
 '(E)-2-nonenal': {'cucumber', 'fat', 'green'},
 '(E)-2-penten-1-ol': {'mushroom'},
 '(E)-2-undecenal': {'fat', 'green', 'soap'},
 '(E)-3-hexenol': {'fresh', 'moss'},
 '(E)-carveol': {'caraway', 'solvent'},
 '(E)-cinnamaldehyde': {'cinnamon', 'paint'},
 '(E)-dihydrocarvone': {'herb', 'warm'},
 '(E)-farnesol': {'muguet'},
 '(E)-isoelemicin'

match molecules from MS data with flavors

In [26]:
flavors = []
mol_flavors = {}
for mol in ms_molecules_ex:
    if mol in mol_to_OD.keys():
        flavors += mol_to_OD[mol]
        mol_flavors[mol] = mol_to_OD[mol]

found_flavors = set(flavors)
found_flavors

{'fat', 'fresh', 'green', 'moss', 'oil', 'sweet'}

mol_flavors contains the molecule from where each OD is coming from 

In [27]:
mol_flavors

{'(E)-3-hexenol': {'fresh', 'moss'},
 '2-dodecenal': {'fat', 'green', 'sweet'},
 'δ-muurolene': {'oil'}}

#### 3. OD ----- Roche Data -----> OQ
Use ontology to find matching OQ

In [68]:
'''def OD_x_OQ():
    return pd.read_csv('./data/DATA_OD-2-OQ.csv', header=None)
    #print(test)
    with open('./data/DATA_OD-2-OQ.csv', 'r', encoding='utf-8') as file:
            reader = csv.reader(file,delimiter=',')
            for row in reader:
                OD = row[0]
                almond = row[1]
                woody = row[2] # usw.
    return test

OD_OQ_df = OD_x_OQ()'''
OD_OQ_df =  pd.read_csv('./data/DATA_OD-2-OQ.csv', header=None)

In [69]:
OD_OQ_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,Acetic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acid-acrid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acidic,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acrid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acrid-acid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Wine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
221,Wine-yeast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
222,Winey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
223,Woody,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
import pandas as pd
import thefuzz 

'''mol_flavors = {'(E)-3-hexenol': {'fresh', 'moss'},
 '2-dodecenal': {'fat', 'green', 'sweet'},
 'δ-muurolene': {'oil'}}'''

OQs = []
ms_OQs = {}
for key, value in mol_flavors.items():
    all_OQS = []
    for index, row in OD_OQ_df.iterrows():
        for keyword in value:
            if fuzz.ratio(keyword, str(row[0]).lower()) >= 60:
                all_OQS.append((row[0]))
                OQs.append(row)
    ms_OQs[key] = all_OQS

OQs = pd.DataFrame(OQs)
OQs


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
132,Mimosa,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
74,Fatty,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,Grape,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
102,Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
118,Leafy-green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
192,Sweat,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193,Sweaty,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194,Sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,Weedy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,Oily,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ms_OQs: dict with mol as keys and OQs as values 

In [175]:
ms_OQs

{'(E)-3-hexenol': ['Mimosa'],
 '2-dodecenal': ['Fatty',
  'Grape',
  'Green',
  'Leafy-green',
  'Sweat',
  'Sweaty',
  'Sweet',
  'Weedy'],
 'δ-muurolene': ['Oily']}

In [179]:
'''OQs = OD_OQ_df[OD_OQ_df.apply(lambda row: any(keyword in str(row).lower() for keyword in found_flavors if fuzz.ratio(keyword, found_flavors)>=15), axis=1)]
OQs
# TODO: e.g. make fat belong to fatty, oil to oily etc. 
# ODs = 'fat', 'fresh', 'green', 'moss', 'oil', 'sweet'
'''

"OQs = OD_OQ_df[OD_OQ_df.apply(lambda row: any(keyword in str(row).lower() for keyword in found_flavors if fuzz.ratio(keyword, found_flavors)>=15), axis=1)]\nOQs\n# TODO: e.g. make fat belong to fatty, oil to oily etc. \n# ODs = 'fat', 'fresh', 'green', 'moss', 'oil', 'sweet'\n"

In [209]:
OQ_descriptors = ["Almond","Woody","Camphor","Leather","Cooked","Spicy","Floral","Fresh","Fruity","Smoky","Cut-grass","Lactonic","Vegetable","Honey","Peel","Sulfurous","Toasty","Vanilla","Green","Vinous","Violet"]

OQs.columns = ["OD"] +OQ_descriptors
selected_OQ_df = OQs.drop("OD",axis = 1)
OQs

,OD,Almond,Woody,Camphor,Leather,Cooked,Spicy,Floral,Fresh,Fruity,...,Lactonic,Vegetable,Honey,Peel,Sulfurous,Toasty,Vanilla,Green,Vinous,Violet
132,Mimosa,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
74,Fatty,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,Grape,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
102,Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
118,Leafy-green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
192,Sweat,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193,Sweaty,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194,Sweet,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,Weedy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,Oily,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
# Get the column names where each row is 1
mol_OQs = {}

for key, value in ms_OQs.items():
    current_OQs = []
    for keyword in value:
        for index, row in OQs.iterrows():
            if row[0] == keyword:
                row_values = OQs.columns[row == 1].tolist()
                current_OQs += row_values
        mol_OQs[key] = set(current_OQs)

mol_OQs


{'(E)-3-hexenol': {'Floral'},
 '2-dodecenal': {'Cut-grass', 'Fruity', 'Green'},
 'δ-muurolene': set()}

In [237]:
# Get the column names where each row is 1
selected_OQs = []
for index, row in selected_OQ_df.iterrows():
    row_values = selected_OQ_df.columns[row == 1].tolist()
    #print(index)
    selected_OQs += row_values
    #mol_OQs

selected_OQs = (set(selected_OQs))
selected_OQs

{'Cut-grass', 'Floral', 'Fruity', 'Green'}

#### 4.1 OQ ------ Binary Matrix -----> OSA


In [75]:
OSA_data = pd.read_table('./data/OSA_dummy.csv', delimiter=';')
OSA_data


,OQ/OSA,floral,fruit,wood,complexity,malt,sweet,peat,dried fruits,aftertase
0,Almond,1,0,0,0,0,0,0,0,0
1,Cooked,0,0,0,1,0,0,0,0,0
2,Cut-grass,0,0,0,0,1,0,0,0,0
3,Floral,1,0,0,0,0,0,0,1,0
4,Fresh,1,0,0,0,0,0,0,1,0
5,Fruity,1,1,0,0,0,1,0,1,0
6,Green,1,0,0,0,0,0,0,1,0
7,Honey,1,0,0,0,0,1,0,1,0
8,Lactonic,0,0,0,0,0,0,0,0,0
9,Leather,0,0,0,0,0,0,0,0,0


In [234]:
mol_OQs

{'(E)-3-hexenol': {'Floral'},
 '2-dodecenal': {'Cut-grass', 'Fruity', 'Green'},
 'δ-muurolene': set()}

In [238]:
mol_OSA ={}

for key, value in mol_OQs.items():
    selected_OSAs = []
    for keyword in value:
        for index, row in OSA_data.iterrows():
            if row[0] in value:
                row_values = OSA_data.columns[row == 1].tolist()
                selected_OSAs += row_values
    mol_OSA[key] = set(selected_OSAs)

mol_OSA


{'(E)-3-hexenol': {'dried fruits', 'floral'},
 '2-dodecenal': {'dried fruits', 'floral', 'fruit', 'malt', 'sweet'},
 'δ-muurolene': set()}

In [239]:
selected_OSAs = []
for index, row in OSA_data.iterrows():
    if row[0] in selected_OQs:
        row_values = OSA_data.columns[row == 1].tolist()
        selected_OSAs += row_values

selected_OSAs = set(selected_OSAs)
selected_OSAs


{'dried fruits', 'floral', 'fruit', 'malt', 'sweet'}

#### normalize intensities ?
#### what to do if 2 molecules have same OSA add intensites?

In [262]:
intensities

[80, 40, 5]

In [263]:
mol_OSA

{'(E)-3-hexenol': {'dried fruits', 'floral'},
 '2-dodecenal': {'dried fruits', 'floral', 'fruit', 'malt', 'sweet'},
 'δ-muurolene': set()}

Flavor profile with OSAs:

dried_fruits: 80 + 40

floral: 80 + 40 

fruit: 40 

malt: 40 

sweet: 40 


#### 4.2 OQ ------ Fuzzy Logic -----> OSA
Use Fuzzy logic to weigh OQ with expert knowledge/Intensity from Masspec


#### 4.3 Comparison BM and FL

#### 5. Show Flavourprofile